### 1. Cloning the Dataset

In [37]:
!rm -r /content/OpenLibrary-Books-Scraper

!git clone https://github.com/AshishJangra27/OpenLibrary-Books-Scraper

!rm -r data
!mkdir data

Cloning into 'OpenLibrary-Books-Scraper'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 110 (delta 8), reused 18 (delta 6), pack-reused 88
Receiving objects: 100% (110/110), 74.74 MiB | 40.49 MiB/s, done.
Resolving deltas: 100% (19/19), done.


### 2. Importing Libraries

In [38]:
import os
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup
from google.colab import drive

for i in os.listdir('/content/OpenLibrary-Books-Scraper/data'):
  os.system('gunzip /content/OpenLibrary-Books-Scraper/data/' + i)

!python /content/OpenLibrary-Books-Scraper/app.py

df = pd.read_csv('data.csv').drop_duplicates('title_id')

del df['category_']

df['book_stats'] = 'Nan'
df['descriptions'] = 'Nan'
df['reading_stats'] = 'Nan'

drive.mount('/content/gdrive/', force_remount=True)

100% 56/56 [00:16<00:00,  3.46it/s]
Mounted at /content/gdrive/


In [39]:
def download(st,en):

  book_stats = []
  descriptions = []
  reading_stats = []

  for ids in tqdm(df['title_id'][st:en]):
    link = 'https://openlibrary.org/' + ids
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      reading_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'readers-stats__wrapper mobile').find_all('li', class_ = 'reading-log-stat')])
    except:
      reading_stat = np.nan
    try:
      book_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'edition-omniline').find_all('div', class_ = 'edition-omniline-item')])
    except:
      book_stat = np.nan
    try:
      description = soup.find('div', class_ = 'book-description').text.replace('\n',' ').strip()
    except:
      description = np.nan

    reading_stats.append(reading_stat)
    book_stats.append(book_stat)
    descriptions.append(description)

  df['book_stats'][st:en]    = book_stats
  df['descriptions'][st:en]  = descriptions
  df['reading_stats'][st:en] = reading_stats

  df[st:en].to_csv('data/' + str(st) + '_' + str(en) + '.csv', index = False)

In [ ]:
start = 0

n = 1000
pr = 5

process = []

for i in range(start,n*pr,n):
  process.append(multiprocessing.Process(target = download, args = ( i , i + n)))

for p in process:
  p.start()

for p in process:
  p.join()


csvs = [csv for csv in os.listdir('data') if '.csv' in csv]
df = pd.DataFrame()
for csv in tqdm(csvs):
    df_ = pd.read_csv('data/' + csv )
    df_['category_'] = csv.split('.csv')[0]
    df = pd.concat((df,df_))

df.to_csv('/content/gdrive/MyDrive/Book Dataset/'+str(start)+'_'+str(i+n)+'.csv' , index = False)

 91%|█████████ | 906/1000 [14:51<01:30,  1.04it/s]